In [24]:
import pathlib
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [25]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [26]:
spark.version

'3.3.2'

In [27]:
data_path = pathlib.Path().absolute() / "data/fhvhv_tripdata_2021-06.csv.gz"
# !ls -lh {data_path}

In [28]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [29]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(data_path.as_posix())

df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/')

In [30]:
parquet_path = pathlib.Path().absolute() / 'data' / 'pq' / 'fhvhv' / '2021' / '06' 
!ls -l --block-size=M {parquet_path}

total 284M
-rw-r--r-- 1 adrian adrian  0M Feb 28 10:56 _SUCCESS
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00000-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00001-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00002-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00003-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00004-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00005-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00006-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10:55 part-00007-ab0cc0c1-4d75-453a-995a-f99b3ea09c5b-c000.snappy.parquet
-rw-r--r-- 1 adrian adrian 24M Feb 28 10

In [31]:
df = spark.read.parquet(parquet_path.as_posix())

**Q3**: How many taxi trips were there on February 15?

In [32]:
from pyspark.sql import functions as F

In [33]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [35]:
df.createOrReplaceTempView('fhvhv_2021_06')

In [37]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



**Q4**: Longest trip for each day

In [38]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [41]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
| 2021-06-22|        91979|
| 2021-06-27|        71931|
| 2021-06-26|        65510|
| 2021-06-23|        59281|
+-----------+-------------+



In [45]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration_hours
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2021-06-25|  66.8788888888889|
| 2021-06-22|25.549722222222222|
| 2021-06-27|19.980833333333333|
| 2021-06-26|18.197222222222223|
| 2021-06-23|16.466944444444444|
| 2021-06-24|13.909722222222221|
| 2021-06-04|             11.67|
| 2021-06-20|10.984444444444444|
| 2021-06-01|           10.2675|
| 2021-06-28| 9.966388888888888|
+-----------+------------------+



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [44]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_02
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()


[Stage 73:>                                                         (0 + 4) / 4]



+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 3233664|
|              B02764|  965568|
|              B02872|  882689|
|              B02875|  685390|
|              B02765|  559768|
+--------------------+--------+




[Stage 74:===================================================>  (189 + 5) / 200]



In [46]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()


[Stage 86:>                                                         (0 + 4) / 4]



+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+




[Stage 87:===========================================>          (161 + 5) / 200]



**Q6**: Most common locations pair

In [47]:
df_zones = spark.read.parquet('zones')

In [49]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [51]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [50]:
df_zones.registerTempTable('zones')

In [57]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_02 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|East New York / E...|   45041|
|Borough Park / Bo...|   37329|
| Canarsie / Canarsie|   28026|
|Crown Heights Nor...|   25976|
|Bay Ridge / Bay R...|   17934|
+--------------------+--------+

